In [152]:
import json

In [153]:
endpoint_namespace = input("enter enpoint namespace: ")
endpoint_route = input("enter enpoint route: ")

endpoint_namespace + '/' + endpoint_route

'therapists/branches/reservations'

In [154]:
route_singular = input("enter singular form of the route: ")
route_singular

'reservation'

In [155]:
route_actions = input('Actions to be added (CRUD): ').upper()
route_actions

'CRUD'

In [156]:
module_name = '::'

for name in endpoint_namespace.split('/'):
  module_name += name.capitalize() + '::'

module_name += endpoint_route.capitalize()

module_name

'::Therapists::Branches::Reservations'

In [157]:
object_params = {}
filter_params = ""
sort_params = ""

object_attributes = []

In [172]:
if 'C' in route_actions or 'U' in route_actions:
  object_params = input('input the payload format of creating or updating the object, as a dictionary: ')
  # object_params must be input in proper format for conversion to json dictionary
  # for ex.: {'transactions': ['method_id', 'category_id', 'type', 'amount', 'memo']}
  # the '' commas around each word are important, or else JSON won't be able to parse
  # Can also use "" instead of ''
  object_params

  print('obj before', object_params)
  object_params = object_params.replace("'", '"')
  object_params = json.loads(object_params)
  print('obj after', object_params)

  if(len(object_params.values()) > 0):
    form_object_attributes = list(object_params.values())[0]

obj before {'reservation': ['branch_id', 'therapist_id', 'memo']}
obj after {'reservation': ['branch_id', 'therapist_id', 'memo']}


In [171]:
list(object_params.values())[0]
object_attributes

':branch_id, :id, :memo, :therapist_id'

In [159]:
if 'R' in route_actions:
  filter_params = input('input the comma-separated filter params: ') # Ex.: :method_id, :type, :category_id, :start_date, :end_date
  sort_params = input('input the comma-separated sort params: ') # Ex. : :amount, :date
  print(filter_params)
  print(sort_params)

:therapist_id, :branch_id
:start_time, :end_time


In [160]:
%run 'controller_generator.ipynb'

cg = ControllerGenerator(
  endpoint_namespace=endpoint_namespace,
  endpoint_route=endpoint_route,
  route_singular=route_singular,
  route_actions=route_actions,
  module_name=module_name,
  object_params=object_params,
  filter_params=filter_params,
  sort_params=sort_params
)

cg.write_to_file()

In [161]:
route_actions

'CRUD'

In [175]:
%run 'use_case_generator.ipynb'

ucg = UseCaseGenerator(
  namespace=endpoint_namespace,
  route=endpoint_route,
  module_name=module_name,
  route_singular=route_singular,
  route_actions=route_actions
)

ucg.add_use_cases()

In [173]:
if 'C' in route_actions or 'U' in route_actions:
  %run 'form_generator.ipynb'

  fg = FormGenerator(
    namespace = endpoint_namespace,
    route_singular = route_singular,
    object_attributes = form_object_attributes
  )

  fg.create_form()